In [4]:
import os
import cv2
import csv
import time
import torch
from datetime import datetime
from ultralytics import YOLO

# OCR (for number plates)
import easyocr

# Fix for OpenMP conflict on macOS (often needed)
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"


def now_ts():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def ensure_csv(csv_path: str):
    if not os.path.exists(csv_path):
        with open(csv_path, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow([
                "Timestamp",
                "IncidentType",      # NUMBER_PLATE / WEAPON / SUSPICIOUS_ACTIVITY
                "LabelOrText",       # plate text or detected class label
                "Confidence",        # detection confidence (0-1)
                "SourceModel",       # plate_model / weapons_model / violence_model
                "x1", "y1", "x2", "y2"
            ])


def append_csv(csv_path: str, row):
    with open(csv_path, "a", newline="") as f:
        csv.writer(f).writerow(row)


def clamp_bbox(x1, y1, x2, y2, w, h):
    x1 = max(0, min(int(x1), w - 1))
    y1 = max(0, min(int(y1), h - 1))
    x2 = max(0, min(int(x2), w - 1))
    y2 = max(0, min(int(y2), h - 1))
    if x2 <= x1:
        x2 = min(w - 1, x1 + 1)
    if y2 <= y1:
        y2 = min(h - 1, y1 + 1)
    return x1, y1, x2, y2


def draw_box(img, x1, y1, x2, y2, text):
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(
        img, text, (x1, max(20, y1 - 8)),
        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
    )


def main():
    # ------------------ CHANGE THESE PATHS ------------------
    PLATE_MODEL_PATH = "license_plate_best.pt"        # YOLO detection model for number plates
    WEAPONS_MODEL_PATH = "weapons_best.pt"    # YOLO detection model for weapons
    VIOLENCE_MODEL_PATH = "violence_non_violence.pt"  # YOLO detection model (your labels: normal/shoplifting/High-suspicion/Low-suspicion)
    # -------------------------------------------------------

    CSV_FILE = "incidents.csv"
    ensure_csv(CSV_FILE)

    # Device selection (Mac M1/M2: "mps", else CPU)
    device = "mps" if torch.backends.mps.is_available() else "cpu"

    # Load models
    plate_model = YOLO(PLATE_MODEL_PATH)
    weapons_model = YOLO(WEAPONS_MODEL_PATH)
    violence_model = YOLO(VIOLENCE_MODEL_PATH)

    # OCR reader (for plate text)
    ocr = easyocr.Reader(["en"], gpu=(device != "cpu"))

    # ------------------ THRESHOLDS (TUNE) -------------------
    PLATE_CONF_TH = 0.35
    WEAPONS_CONF_TH = 0.50
    SUSPICION_CONF_TH = 0.50
    # -------------------------------------------------------

    # Cooldowns to avoid logging every frame
    PLATE_COOLDOWN_SEC = 3.0
    WEAPON_COOLDOWN_SEC = 3.0
    SUSPICION_COOLDOWN_SEC = 5.0

    # Track last logged time per event key
    last_logged = {}  # key -> time.time()

    # Start webcam
    cap = cv2.VideoCapture(0)
    print("--- Integrated Incident Logger (Plate + Weapons + Suspicious Activity). Press 'q' to quit ---")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        H, W = frame.shape[:2]
        vis = frame.copy()  # frame for drawing overlays

        # =====================================================
        # 1) NUMBER PLATE DETECTION + OCR
        # =====================================================
        plate_results = plate_model(frame, device=device, verbose=False)

        for r in plate_results:
            for box in r.boxes:
                det_conf = float(box.conf[0])
                if det_conf < PLATE_CONF_TH:
                    continue

                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = clamp_bbox(x1, y1, x2, y2, W, H)

                crop = frame[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                # OCR the cropped plate area
                plate_text = ""
                ocr_out = ocr.readtext(crop)
                if ocr_out:
                    best = max(ocr_out, key=lambda t: float(t[2]))
                    plate_text = (best[1] or "").strip()

                # Cooldown key: prefer plate_text if we got it
                key = f"plate:{plate_text}" if plate_text else f"plate_bbox:{x1},{y1},{x2},{y2}"
                tnow = time.time()
                if (key not in last_logged) or (tnow - last_logged[key] >= PLATE_COOLDOWN_SEC):
                    last_logged[key] = tnow
                    append_csv(CSV_FILE, [
                        now_ts(),
                        "NUMBER_PLATE",
                        plate_text if plate_text else "UNREADABLE",
                        f"{det_conf:.2f}",
                        "plate_model+easyocr",
                        x1, y1, x2, y2
                    ])
                    print(f"[LOG] NUMBER_PLATE: {plate_text if plate_text else 'UNREADABLE'} (conf={det_conf:.2f})")

                draw_box(vis, x1, y1, x2, y2, f"PLATE {det_conf:.2f} {plate_text[:12]}")

        # =====================================================
        # 2) WEAPONS DETECTION (YOLO detect)
        # =====================================================
        weapons_results = weapons_model(frame, device=device, verbose=False)

        for r in weapons_results:
            names = getattr(r, "names", None) or weapons_model.names
            for box in r.boxes:
                conf = float(box.conf[0])
                if conf < WEAPONS_CONF_TH:
                    continue

                class_id = int(box.cls[0])
                label = names.get(class_id, str(class_id)) if isinstance(names, dict) else str(class_id)

                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = clamp_bbox(x1, y1, x2, y2, W, H)

                key = f"weapon:{label}"
                tnow = time.time()
                if (key not in last_logged) or (tnow - last_logged[key] >= WEAPON_COOLDOWN_SEC):
                    last_logged[key] = tnow
                    append_csv(CSV_FILE, [
                        now_ts(),
                        "WEAPON",
                        label,
                        f"{conf:.2f}",
                        "weapons_model",
                        x1, y1, x2, y2
                    ])
                    print(f"[LOG] WEAPON: {label} (conf={conf:.2f})")

                draw_box(vis, x1, y1, x2, y2, f"WEAPON {label} {conf:.2f}")

        # =====================================================
        # 3) "VIOLENCE" MODEL (YOUR OUTPUT IS DETECT):
        #    {0:'normal',1:'shoplifting',2:'High-suspicion',3:'Low-suspicion'}
        #    We log everything except 'normal'.
        # =====================================================
        suspicion_results = violence_model(frame, device=device, verbose=False)

        for r in suspicion_results:
            names = getattr(r, "names", None) or violence_model.names
            for box in r.boxes:
                conf = float(box.conf[0])
                if conf < SUSPICION_CONF_TH:
                    continue

                class_id = int(box.cls[0])
                label = names.get(class_id, str(class_id)) if isinstance(names, dict) else str(class_id)

                # Ignore "normal"
                if str(label).strip().lower() == "normal":
                    continue

                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = clamp_bbox(x1, y1, x2, y2, W, H)

                key = f"suspicion:{label}"
                tnow = time.time()
                if (key not in last_logged) or (tnow - last_logged[key] >= SUSPICION_COOLDOWN_SEC):
                    last_logged[key] = tnow
                    append_csv(CSV_FILE, [
                        now_ts(),
                        "SUSPICIOUS_ACTIVITY",
                        label,
                        f"{conf:.2f}",
                        "violence_model(detect)",
                        x1, y1, x2, y2
                    ])
                    print(f"[LOG] SUSPICIOUS_ACTIVITY: {label} (conf={conf:.2f})")

                draw_box(vis, x1, y1, x2, y2, f"SUS {label} {conf:.2f}")

        # Show the live feed
        cv2.imshow("Integrated Incident Logger", vis)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>

In [7]:
import os
import ssl
import certifi
import urllib.request
from ultralytics import YOLO
import easyocr
import cv2

# ---------------------------
# SSL FIX (Safe)
# ---------------------------
try:
    ssl._create_default_https_context = ssl.create_default_context(cafile=certifi.where())
    print("SSL configured using certifi ✔")
except Exception as e:
    print("Certifi SSL setup failed:", e)

# ---------------------------
# OPTIONAL SSL BYPASS (Unsafe)
# ---------------------------
try:
    ssl._create_default_https_context = ssl._create_unverified_context
    print("⚠ SSL verification bypassed (unsafe, but works)")
except Exception as e:
    print("SSL bypass failed:", e)

# ---------------------------
# PATHS (Change if needed)
# ---------------------------
YOLO_MODEL_PATH = "yolov8n.pt"   # put your local YOLO model here
IMAGE_PATH = "test.jpg"          # image for testing
EASYOCR_MODEL_DIR = "./easyocr_models"  # offline EasyOCR folder (optional)

# ---------------------------
# LOAD YOLO MODEL
# ---------------------------
print("Loading YOLO model...")
if not os.path.exists(YOLO_MODEL_PATH):
    print("YOLO model not found locally. It will try downloading...")
yolo_model = YOLO(YOLO_MODEL_PATH)
print("YOLO loaded ✔")

# ---------------------------
# LOAD EASYOCR
# ---------------------------
print("Loading EasyOCR...")

reader = None

try:
    # Try ONLINE first
    reader = easyocr.Reader(['en'], gpu=False)
    print("EasyOCR loaded ONLINE ✔")
except Exception as e:
    print("EasyOCR online load failed:", e)
    print("Switching to OFFLINE mode...")

    try:
        reader = easyocr.Reader(
            ['en'],
            gpu=False,
            model_storage_directory=EASYOCR_MODEL_DIR,
            download_enabled=False
        )
        print("EasyOCR loaded OFFLINE ✔")
    except Exception as e2:
        print("❌ EasyOCR offline load also failed.")
        raise e2

# ---------------------------
# RUN DETECTION
# ---------------------------
print("Running YOLO detection...")
results = yolo_model(IMAGE_PATH)

# Draw boxes
img = cv2.imread(IMAGE_PATH)

for r in results:
    for box in r.boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        crop = img[y1:y2, x1:x2]

        # OCR on detected region
        ocr_result = reader.readtext(crop)

        for (bbox, text, prob) in ocr_result:
            print("Detected Text:", text)

        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Save output
cv2.imwrite("output.jpg", img)
print("Done ✔ → output saved as output.jpg")

SSL configured using certifi ✔
⚠ SSL verification bypassed (unsafe, but works)
Loading YOLO model...
YOLO model not found locally. It will try downloading...
WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.4.0/yolov8n.pt... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>


######################################################################## 100.0%
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


YOLO loaded ✔
Loading EasyOCR...
EasyOCR online load failed: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>
Switching to OFFLINE mode...
❌ EasyOCR offline load also failed.


FileNotFoundError: Missing ./easyocr_models/craft_mlt_25k.pth and downloads disabled